## Notebook for reconstructing magnetisation from a single aligned tilt-series

### 1. 3D mask adjustments
### 2. Reconstruction
### 3. Inspection
### 4. Diagnostics

#### Imports

In [2]:
%matplotlib qt

In [3]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import pickle
import os

In [4]:
from timeit import default_timer as timer
import hyperspy.api as hs
import skimage.filters as skfl

In [5]:
#load editable developement pyramid version
import sys

pyramid_dev_version_path=os.path.abspath(r"D:\Programmes\Git\empyre-patched\empyre")
sys.path.append(pyramid_dev_version_path)

In [6]:
import pyramid as pr
import mbir.alignment as pa
import mbir.util as pu
import mbir.reconstruction as pre
import mbir.diagnostics as pd

### 1. Mask adjustments 
#### Load and inspect the data

In [7]:
result_folder=r".\results"
results={}

with open(result_folder+r"\data_series.pickle", "rb") as f: 
    data_series=pickle.load(f)

In [ ]:
data_series.plot_mask()
dimz, dimy, dimx = data_series.dim
pu.matshow_n([np.sum(data_series.mask,axis=0), np.sum(data_series.mask,axis=1), np.sum(data_series.mask,axis=2)],
         ["z sum","y sum", "x sum"])
print("Pixel spacing: %.3f nm"%data_series.a)
print("Reconstruction dimensions (z,y,x):",data_series.dim)

#### Optimise mask position to speed up reconstruction and include boundary charges

In [ ]:
last_valid_x_slice=10 # how far from the left edge the model starts being correct
x_extension=1 # how much boundary charges to add.
auto_centre=True

data_refined, mask_edge = pre.translate_trim_data_series(data_series, auto_centre=auto_centre, x_extension = x_extension, 
                               last_valid_x_slice = last_valid_x_slice, plot_results=True, subcount=1 )

In [ ]:
#save a backup of the refined dataset

with open(result_folder+r"\data_refined.pickle", "wb") as f:
    pickle.dump(data_refined, f)

print("mask_edge:", mask_edge)

### 2. Reconstruction

In [49]:
#load the backup

with open(result_folder+r"\data_refined.pickle", "rb") as f:
    data_refined=pickle.load(f)
    mask_edge=1

In [50]:
# To obtain the true minimum, the Conjugate-gradient solver needs to run for as many iterations as there are free variables. 
# Usually convergence is achieved much faster.
print("number of free variables:",len(data_refined.mask[data_refined.mask])*3)

number of free variables: 44970


#### Perform the reconstruction by regularising the variance and the Heisenberg exchange energy

In [51]:
# Do the reconstruction
iterations=5000
regulariser_strengths = (1e-1,1e0) # (variance coeff, exchange energy coeff)

data=data_refined
mag_guess = None
#x-axis edge of boundary charges
regulariser_mask=data.mask.copy()
regulariser_mask[:,:,:mask_edge]=False #do not regularise the amplitude of the boundary charges


result_note=f"{iterations} iterations"
cost_values=[]

start_time = timer()

magdata_rec, cost_function = pre.reconstruct_from_phasemaps(data, lam=regulariser_strengths, 
                                    max_iter=iterations, ramp_order=1, verbose=True, plot_input=False, plot_results=False, 
                                   regulariser_type='amplitude',mag_0=mag_guess, reg_mask=regulariser_mask)
pre.append_valid_costfunction_values(cost_values, cost_function)
print(f"Finished run with cost {cost_function.chisq}")
    
end_time = timer()
execution_time = end_time - start_time #seconds
print(f"calculating {iterations} iterations took {execution_time/60:.1f} minutes")

    
result = (magdata_rec, cost_function, cost_values, result_note)
results[regulariser_strengths] = result

Regularising amplitude variance and exchange energy


CG:   0%|          | 0/5000 [00:00<?, ?it/s]

Finished run with cost 17993.23691508134
calculating 5000 iterations took 12.2 minutes


In [ ]:
# save result for later analysis
with open(result_folder+r"\reconstruction_results.pickle","wb") as f:
    pickle.dump(results, f)

### 3. Inspect the reconstruction

In [16]:
# load previous results
with open(result_folder+r"\reconstruction_results.pickle","rb") as f:
    results=pickle.load(f)
    (magdata_rec, cost_function, cost_values, result_note)=results[(1e-1,1e0)]
with open(result_folder+r"\data_refined.pickle", "rb") as f:
    data_refined=pickle.load(f)
    mask_edge=1
    

#### Inspect the magnetisation magnitude, direction, and max spin angle.

In [52]:
angle_field = pre.inspect_magdata(magdata_rec[:,:,:,mask_edge:], plot_angles=True, ar_dens=2)
pre.inspect_cost_values(cost_values, print_chis=True)
plt.matshow(np.max(angle_field.field,axis=0), vmax=90, vmin=0, origin='lower')
plt.title("maximum spin angle")
plt.colorbar()

Max spin angle: 177.14386169182103

N = 1 cost value pairs.
model, regulariser, sum
1.75897e+04; 4.03503e+02; 1.79932e+04



#### Inspect difference between simulated and measured phase maps

In [17]:
t=pre.simulate_reconstruction(data_refined, magdata_rec, cost_function)

D:\Programmes\Git\empyre-patched\empyre\pyramid\phasemap.py:698: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(figsize=figsize)


### 4.Diagnostics

### 4.1 Histogram of magnetisation magnitudes

In [43]:
t=pd.histogram_magnetisation (magdata_rec[:,:,:,mask_edge:], [0,2], save_img=False, fit_gauss=False)

bin_size: 0.02 T
mean and std: 0.6144624374212183 0.24518120264436266


### 4.2 Optimal estimation diagnostics

#### First, select voxel to evaluate

In [25]:
#select the voxel to evaluate
voxel_position = pd.find_voxel_coords(data_refined.mask, position_zyx=[None,50,45], mask_edge=mask_edge, plot_results=True)

#### Perform an estimation of the hessian matrix to be used in optimal estimation diagnostics

In [26]:
mag_rec = magdata_rec
cost_f = cost_function
data = data_refined

diagnostic_results = pd.bayesian_diagnostics(data, mag_rec, cost_f, voxel_position_zyx=voxel_position, 
                       verbose=True, max_iter=200, plot_results=True)


finished calculating for x component
finished calculating for y component
finished calculating for z component

Voxel position: (7, 50, 45)
Magnetisation vector is:
        (M_x = 1.16e-01 +/- 3.64e-02 T,
         M_y = 5.48e-01 +/- 1.32e-01 T,
         M_z = -3.72e-03 +/- -7.30e-04 T)
Amplitude: 0.560 +/- 0.137 T
Spatial resolution (dx, dy, dz): 3.1, 6.2, 5.3 pixels
Pixel spacing: 10.27 nm


In [38]:
diagnostic_results

(0.1366004960793148,
 array([3.05423261, 6.22208682, 5.32201192]),
 (<pyramid.diagnostics.Diagnostics at 0x1671fffef40>,
  <pyramid.diagnostics.Diagnostics at 0x1671fff3bb0>))

#### Average Full-Width-Half-Maximum of the 3D point spread function introduced by the reconstruction

In [32]:
# get average FWHM in nm
print("spatial resolution FWHM, nm")
np.sqrt(np.average(diagnostic_results[1]**2))*data_refined.a

spatial resolution FWHM, nm


51.79399600313342

#### Average saturation magnetisation *mu0

In [33]:
# average magnetisation *mu0
mag_avg=np.mean(magdata_rec.field_amp[magdata_rec.get_mask()])
print("average magnetisation, T")
mag_avg

average magnetisation, T


0.6289614735057029

#### Magnetisation magnitude error due to image missalignment

In [35]:
# evaluate perturbation of forcing one phase image to be perfect
error_sum=diagnostic_results[0]
err_alignment = error_sum/data_refined.count
print("relative alignment error, %, ", err_alignment/mag_avg*100)
err_alignment

relative alignment error, %,  2.4131578064874946


0.015177832897701644

#### Magnetisation magnitude error due to measurement noise

In [40]:
# estimage perturbation on reconstruction
std=0.074 #standard deviation of phase image noise

diag_x=diagnostic_results[2][0]
G=diag_x.gain_row
err_meas = np.sqrt(np.dot(G,G))*std
print("relative phase measurement error, %, ", err_meas/mag_avg*100)
err_meas

relative phase measurement error, %,  0.7937158413892129


0.004992166851449781

#### Total precision of magnitude measurement

In [44]:
# total magnitude error in T from phase perturbations
err_pert = np.hypot(err_alignment,err_meas)
print("relative phase perturbation error, %, ", err_pert/mag_avg*100)
err_pert

relative phase perturbation error, %,  2.540337661785087


0.0159777451895838

### 4.3 Fourier shell correlation

In [53]:
#FSC
#do for all sub-components:
mag_rec = magdata_rec[:,:,:,mask_edge:]

field_array=mag_rec.field.copy()
field_array[:,:,:,:mask_edge]=0

FSCs = []
resolutions=[]
for i in range(3):
    array_3d=field_array[i,:,:,:]
    ffts = pd.fsc_split_array(array_3d)
    freq, FSC, ns_effective = pd.fsc_calculate_correlation(array_3d, *ffts, scale=2, plot_results=False)
    FSCs.append(FSC)

In [54]:
sne=np.sqrt(ns_effective)
sigma_3 = 3/(sne+2)
hbit=((0.2071+1.9102/sne)/(1.2071+0.9102/sne)) # half-bit criterion

freq_p=2*np.array(freq) #for plotting convert to f/nyquist

def find_intersection(FSC, hbit, freq_p):
    for i in range(len(FSC))[10:]:
        fi=FSC[i]
        thi=[hbit[i]]
        if thi>fi:
            resolution = tick_function([freq_p[i-1]])[0]
            return(resolution)
            break
            
def tick_function(f):
    freqs=[]
    for freq in f:
        if freq<=1e-10:
            freqs.append(None)
        else:
            f=1/freq*mag_rec.a
            freqs.append("%.1f"%f)
    return (freqs) #in nm

plt.figure()
ax1 = plt.gca()
ax2 = ax1.twiny()

s='xyz'
fmts=["r.-","g.-","b.-"]
for i,FSC in enumerate(FSCs):
    resolution = find_intersection(FSC, hbit, freq_p)
    
    ax1.plot(freq_p, FSC, fmts[i], label=f"$m_{s[i]}$, resolution $= {resolution}$ nm")

ax1.plot(freq_p, hbit, 'k-', label="1/2 bit threshold")
#ax1.plot(freq_p, sigma_3, 'k-.', label="3 sigma")
ax1.set_xlabel("Spatial frequency / Nyquist")
ax1.legend(loc='lower left')
ax1.set_ylabel("Correlation coefficient")


xtickslocs = ax1.get_xticks()[1:-1]
ax2.set_xlim(ax1.get_xlim())
ax2.set_xticks(xtickslocs)
ax2.set_xticklabels(tick_function(xtickslocs))
ax2.set_xlabel("Resolution, nm")
plt.tight_layout()
# plt.savefig("FSC.png",dpi=200)


#### End of notebook. We now have a 3D magnetisation reconstruction and the error analysis.